In [32]:
# import dependenices.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import requests
from citipy import citipy
from config import weather_api_key
from datetime import datetime

In [18]:
# Create a new set of 2,000 random latitudes and longitudes.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [19]:
#previww coordinates and structure.
coordinates = list(lat_lngs)
print(coordinates[0:3])

[(-63.81992619034554, -167.12058181664182), (-67.97833870558419, -7.370793369353208), (81.95396891723234, -138.44594850714157)]


In [28]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

735

In [29]:
# preview city data 
cities[0:5]

['avarua', 'cape town', 'tuktoyaktuk', 'jamestown', 'ushuaia']

In [30]:
# create endpoint url API looping
url = 'http://api.openweathermap.org/data/2.5/weather?&units=imperial&appid=' + weather_api_key 

In [59]:
r = requests.get(url + "&q=" + cities[13].replace(" ","+") ).json()

In [81]:
r

{'coord': {'lon': 140.77, 'lat': -37.83},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 48.2,
  'feels_like': 43.65,
  'temp_min': 48.2,
  'temp_max': 48.2,
  'pressure': 1022,
  'humidity': 87},
 'visibility': 10000,
 'wind': {'speed': 5.82, 'deg': 300},
 'clouds': {'all': 90},
 'dt': 1604606082,
 'sys': {'type': 1,
  'id': 9560,
  'country': 'AU',
  'sunrise': 1604604276,
  'sunset': 1604654190},
 'timezone': 37800,
 'id': 2156643,
 'name': 'Mount Gambier',
 'cod': 200}

In [92]:
# Create an empty list to hold the weather data.
city_data = []
skip_count = 0
# Loop through all the cities in the list.
for i, city in enumerate(cities):
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_description = city_weather['weather'][0]['description']
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_weather_description})

# If an error is experienced, skip the city.
    except:
        skip_count += 1
        continue
# print api summary notes.
print('{} cities skipped.'.format(skip_count)) 
print('data for {} cities acquired '.format(len(city_data)))

60 cities skipped.
data for 675 cities acquired 


In [96]:
# review dataframe
city_data_df = pd.DataFrame(city_data)
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Avarua,CK,-21.21,-159.78,75.20,78,86,17.87,overcast clouds
1,Cape Town,ZA,-33.93,18.42,66.99,88,75,9.17,light rain
2,Tuktoyaktuk,CA,69.45,-133.04,6.80,77,90,13.87,light snow
3,Jamestown,US,42.10,-79.24,64.00,42,1,11.41,clear sky
4,Ushuaia,AR,-54.80,-68.30,50.00,46,75,13.87,broken clouds


In [97]:
# Export the City_Data into a CSV.
city_data_df.to_csv('WeatherPy_Database.csv',index_label='City_ID')